In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
from dmet.ccsd_solver import CCSDSolver
import time
from pyscf import gto
from utils import rel_err
import os, psutil

In [3]:
co2 =  [["C", np.array([17.0000, 10.0000, 0.0000 ])],
        ["O", np.array([18.0221, 10.5679, 0.0000 ])],
        ["O", np.array([15.9779, 9.4321, 0.0000  ])]] 

# Normalize CO2
d_co1 = np.linalg.norm(co2[0][1] - co2[1][1])
d_co2 = np.linalg.norm(co2[0][1] - co2[2][1])

ev = 27.2114
min_dist = 0.985944

CO2 = f"C 0.0 0.0 0.0 ; O -{d_co1} 0.0 0.0 ; O {d_co2} 0.0 0.0"
Mn = f"Mn {d_co2 + min_dist} 0.0 0.0"
geometry = f"{CO2} ; {Mn}"

geometry

'C 0.0 0.0 0.0 ; O -1.1692727739924489 0.0 0.0 ; O 1.1692727739924502 0.0 0.0 ; Mn 2.1552167739924504 0.0 0.0'

In [4]:
mol = gto.Mole() # Instantiate the molecule class in PySCF
mol.atom = geometry
mol.basis = "ccpvdz" # Use "minao" as the basis set
mol.charge = 2 # Assign the charge of the molecule
mol.spin = 5  # Assign the spin of the molecule
mol.build() # Build the molecule object

In [5]:
freezer = []
isum = 0
isum2 = -1

itemp = 0
isum2 += 4

for total_basis in mol.spheric_labels():
    item = total_basis.split()
    item0 = int(item[0])
    if ( ( item0 >= isum ) and ( item0 <= isum2 ) ):
        orbital = int(item[2][0])
        shell = item[2][1]

        if (item[1] == 'C') or (item[1] == 'O'):
            if orbital != 2:
                freezer.append(itemp)
        elif item[1] == 'Mg':
            if orbital != 3:
                freezer.append(itemp)
            if shell != 's':
                freezer.append(itemp)
        elif item[1] == 'Cu':
            if (orbital != 3) and (orbital != 4):
                freezer.append(itemp)
            if (orbital == 4) and shell != 's':
                freezer.append(itemp)
        itemp+=1

In [6]:
len(freezer)

30

In [7]:
actual_energy = -36397.363860

## Frozen orbitals

In [8]:
initial_mem = psutil.Process(os.getpid()).memory_info().rss

In [9]:
ccsd_solver = CCSDSolver()

s = time.time()
energy = ccsd_solver.simulate(mol, frozen = freezer)
e = time.time()
print('Duration:', e - s)
print(energy)

Number of orbitals:  85
Frozen orbitals:  [0, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 34, 35, 36, 37, 38, 39, 40, 41]
CCSD time:  2.3209068775177
Duration: 4.9778501987457275
-1335.2170529652988


In [10]:
final_mem = psutil.Process(os.getpid()).memory_info().rss
final_mem - initial_mem

116760576

In [11]:
total_energy = energy
print(f'Actual total energy: {actual_energy:.12f} Ha\n')
print(f"Total energy: {total_energy:.12f} Ha")
print(f"Total energy ev: {total_energy*ev:.12f} ev")

rel_error = rel_err(actual_energy, total_energy*ev)
print(f'Relative error: {rel_error:.12f}')

Actual total energy: -36397.363859999998 Ha

Total energy: -1335.217052965299 Ha
Total energy ev: -36333.125315059930 ev
Relative error: 0.001764923009


## No frozen orbitals

In [12]:
initial_mem = psutil.Process(os.getpid()).memory_info().rss

In [13]:
ccsd_solver = CCSDSolver()

s = time.time()
energy = ccsd_solver.simulate(mol)
e = time.time()
print('Duration:', e - s)
print(energy)

Number of orbitals:  85
Frozen orbitals:  0
CCSD time:  80.13259768486023
Duration: 82.89638328552246
-1336.1850464982776


In [14]:
final_mem = psutil.Process(os.getpid()).memory_info().rss
final_mem - initial_mem

315662336

In [15]:
total_energy = energy
print(f'Actual total energy: {actual_energy:.12f} Ha\n')
print(f"Total energy: {total_energy:.12f} Ha")
print(f"Total energy ev: {total_energy*ev:.12f} ev")

rel_error = rel_err(actual_energy, total_energy*ev)
print(f'Relative error: {rel_error:.12f}')

Actual total energy: -36397.363859999998 Ha

Total energy: -1336.185046498278 Ha
Total energy ev: -36359.465774283235 ev
Relative error: 0.001041231608
